In [ ]:
### Dataset and Sampling
--------------------------

MovieLens 20M Dataset: 20,000,000 ratings from 138,000 users on 27000 movies. <br>
https://grouplens.org/datasets/movielens/20m/

We split the 20M dataset into five separate datasets with 4M ratings each ( split_1.csv to split_5.csv).
Split_5.csv is held out as the final testset and the remaining 4 splits are used for training and tuning

In [ ]:
# Objective
------------
Our goal is to build a system that recommends movies that are highly relevant to a given user as well as, based on the user's rating history. 
In order to achieve this, our system must accurately predict the unrated items for all users as well as display items that
the user has high interest in.
A highly accurate system allows us to recommend relevant movies by simply suggesting the top k highest predictions for any given user. 
Therefore, our business objective will be to optimize our models based on the lowest root-mean-squared error(RMSE). 
Our baseline benchmark will be a random predicted rating. That is, we are comparing our model with 
We will also be comparing the performance of our model 


# Models & Methodology
----------------------
Below is an overview of each model we built, as well as the methodology we used in constructing said model.

<br>
### Content-based Approach: <br>
The content-based approach attempts to dissect each item into a list of features that best describes that given item. 
Then, we try to describe each user using these same features by analyzing the previous rating history of that user. 
For example, if a given user consistently gives romance films high ratings, then we can deduce that the user is 
a fan of romance films and the user's model will reflect that.
Next, we compute a predicted rating for a given user and a given item by examining how closely their features
are aligned. We will compute a predicted rating for every user and item pair that the user has not already rated.
Finally, we can sort these predicted ratings from greatest to least and recommend the top k items to a user. 


Our process and methodology in constructing the content-based model is detailed below: <br>

**1) Determine desired features for user/item vectors. <br>**
During this initial step, we wanted to include features that were important descriptors of movies, but can also be used to
describe the users.
We ultimately decided that the primary features we would include were: genre, cast, director, and IMDB rating.
Since we could not include every actor/actress and director for every film, we narrowed the cast and director features
down to around the top 800 people for these two categories.
This was done by selecting the highest grossing actors/actresses and directors.

**2) Extract features for each movie in dataset from IMDB API and build item profiles. <br>**
We utilized the IMDbPY API to extract genre, cast, and director data for all 27,000 movies.
We then used the IMDB data to build the item feature vector for each movie.
For example, if a given movie was designated as an action film on IMDB, then a "1" was placed in the matrix entry 
corresponding to the feature "Action". For every category that the movie does not belong to, a "0" was placed in the
matrix entry. This process was done for every feature and every movie in our dataset.

**3) Build user profiles for each user in the dataset. <br>**


**4) Combine item profiles and user profiles into the same feature space to then compute similarity metric. <br>**
After creating all item profiles and user profiles, we then compute the similarity metric (i.e. cosine similarity) between
every user vector and every item vector that the user has not already interacted with. This similarity metric will then 
be rescaled to a value between 0.5 and 5.0, which will then represent the models's predicted rating for a user toward
a specific item.

**5) Recommend the top k (i.e. 10) items to user. <br>**
Sort predicted ratings from highest to lowest and recommend the top k items to the user. If the user consumes an item
and provides feedback, then the process restarts from step 3.

Note: This practice of predicting every missing element in the user-item matrix and recommending the top k
items is often too computationally expensive. This matrix is very large and requires constant updating whenever 
there are any changes to the user profiles. To address this issue, we would suggest only computing/recomputing the 
missing matrix values periodically (i.e. once a day, week, month) or using locality-sensitive hashing as described below.

<br>
### Baseline Approach and Results: <br>
This is our baseline model where $\hat{r}_{ui} = b_{ui} = \mu + b_u + b_i$.<br>

In one of the KNN algorithms, KNNBaseline, we can take into account a baseline rating. We therefore did the grid search on baselines to find the optimal combination of hyper parameters giving the best performance and used the testing dataset to check the accuracy of our tuned model.

ALS (RMSE: 94.33%) 
n_epochs: 100 
reg_i: 5
reg_u: 5


### LSH Cosine (sim hash): <br>

Here we implement the KNN based models, but using only the approximate nearest neighbors. We used two different variations of KNN. They are KNN Basic, KNN Baseline.  
Notation: <br>

k: The maximum number of neighbors to take into account <br>
$ N_i^k(u) $ : The set consisting of at most k neighbors of user u who have rated item i <br>
$ N_u^k(i) $ : The set consisting of at most k neighbors of item i rated by user u <br>
$ sim(u,v) $ : Similarity of user u to user v <br>
$ sim(i,j) $ : Similarity of item i to item j <br>

We used the following similarity metrics: Cosine Similarity, Pearson Correlation with Baseline Similarity.

#### Cosine similarity: <br>
$
    \text{cosine_sim}(u, v) = \frac{
\sum\limits_{i \in I_{uv}} r_{ui} \cdot r_{vi}}
{\sqrt{\sum\limits_{i \in I_{uv}} r_{ui}^2} \cdot
\sqrt{\sum\limits_{i \in I_{uv}} r_{vi}^2}
}
$

#### Pearson Baseline similarity: <br>
$
    \text{pearson_baseline_sim}(u, v) = \hat{\rho}_{uv} = \frac{
    \sum\limits_{i \in I_{uv}} (r_{ui} -  b_{ui}) \cdot (r_{vi} -
    b_{vi})} {\sqrt{\sum\limits_{i \in I_{uv}} (r_{ui} -  b_{ui})^2}
    \cdot \sqrt{\sum\limits_{i \in I_{uv}} (r_{vi} -  b_{vi})^2}}
$

Accuracy Metric used: RMSE (Root Mean Squared Error)

### i) KNN Basic: <br>
$\hat{r}_{ui}$ : Predicted rating of user u for item i <br>
User Based: <br>
$ \hat{r}_{ui} = \frac{
\sum\limits_{v \in N^k_i(u)} \text{sim}(u, v) \cdot r_{vi}}
{\sum\limits_{v \in N^k_i(u)} \text{sim}(u, v)} $ 

### ii) KNN Baseline:

Here we take into account the baseline ratings. <br> 

User Based:

$ \hat{r}_{ui} = b_{ui} + \frac{ \sum\limits_{v \in N^k_i(u)}
\text{sim}(u, v) \cdot (r_{vi} - b_{vi})} {\sum\limits_{v \in
N^k_i(u)} \text{sim}(u, v)} $



Our results from testing are exhibited below.

## Sim-Hash (Cosine similarity):

We refer to the following paper for cosine similarity hashing (http://www.cs.princeton.edu/courses/archive/spr04/cos598B/bib/CharikarEstim.pdf). <br>

1) create a random vector of size number of movies having values -1,1 (using a normal distribution) <br>
2) calculate the dot product with the item rating vector of the user and if dot product is >0 output 1 else output 0 <br>
3) repeat this for b bands and r rows <br>

If we see the distribution of similarity of a user with all other users and number of movies in common with other users, we see an expected behaviour. So we use a high threshold range of (0.8,0.9) for identifying similar users.
(refer to user_data_plot.ipynb)

In [1]:
from IPython.display import Image
Image(filename='plots/120228.png')

IOError: [Errno 2] No such file or directory: u'plots/120228.png'

In [2]:
from IPython.display import Image
Image(filename='plots/120228_size.png')

IOError: [Errno 2] No such file or directory: u'plots/120228_size.png'

We try different band(b)(also known as the hash size) and row(r)(also known as the number of hash tables) values and using the thershold t in (0.8,0.83,0.85,0.87,0.9), calculate the precision and recall values of the approximate nearest neighbors of a user from the r rows. (code is in the file complete_run.py).

b: (2,3,4,5,6,7,8,9,10)<br>
r: (4,6,8,10,12,14,16,18,20)<br>
threshold: (0.8,0.83,0.85,0.87,0.9)

the b-r table with recall/precision values for the threshold 0.87 is <br>
code for reading the results is in read_br_results.ipynb <br>

<table class="table table-bordered table-hover table-condensed">
<thead><tr><th title="Field #1">b-values\r-values</th>
<th title="Field #2">4</th>
<th title="Field #3">6</th>
<th title="Field #4">8</th>
<th title="Field #5">10</th>
<th title="Field #6">12</th>
<th title="Field #7">14</th>
<th title="Field #8">16</th>
<th title="Field #9">18</th>
<th title="Field #10">20</th>
</tr></thead>
<tbody><tr>
<td align="right">2</td>
<td>0.721/0.643</td>
<td>0.876/0.636</td>
<td>0.927/0.632</td>
<td>0.963/0.632</td>
<td>0.976/0.629</td>
<td>0.986/0.629</td>
<td>0.994/0.628</td>
<td>0.997/0.628</td>
<td>0.998/0.627</td>
</tr>
<tr>
<td align="right">3</td>
<td>0.442/0.645</td>
<td>0.611/0.658</td>
<td>0.702/0.643</td>
<td>0.776/0.643</td>
<td>0.827/0.636</td>
<td>0.876/0.635</td>
<td>0.917/0.637</td>
<td>0.941/0.634</td>
<td>0.954/0.633</td>
</tr>
<tr>
<td align="right">4</td>
<td>0.252/0.653</td>
<td>0.383/0.655</td>
<td>0.481/0.665</td>
<td>0.551/0.663</td>
<td>0.655/0.664</td>
<td>0.666/0.649</td>
<td>0.708/0.65</td>
<td>0.758/0.646</td>
<td>0.79/0.647</td>
</tr>
<tr>
<td align="right">5</td>
<td>0.149/0.669</td>
<td>0.221/0.679</td>
<td>0.267/0.651</td>
<td>0.347/0.695</td>
<td>0.367/0.659</td>
<td>0.44/0.674</td>
<td>0.477/0.667</td>
<td>0.485/0.652</td>
<td>0.541/0.658</td>
</tr>
<tr>
<td align="right">6</td>
<td>0.082/0.68</td>
<td>0.109/0.682</td>
<td>0.137/0.67</td>
<td>0.183/0.679</td>
<td>0.21/0.681</td>
<td>0.239/0.671</td>
<td>0.3/0.689</td>
<td>0.328/0.687</td>
<td>0.328/0.673</td>
</tr>
<tr>
<td align="right">7</td>
<td>0.043/0.709</td>
<td>0.063/0.691</td>
<td>0.082/0.686</td>
<td>0.112/0.706</td>
<td>0.136/0.699</td>
<td>0.15/0.701</td>
<td>0.163/0.697</td>
<td>0.197/0.71</td>
<td>0.202/0.697</td>
</tr>
<tr>
<td align="right">8</td>
<td>0.021/0.695</td>
<td>0.037/0.712</td>
<td>0.052/0.727</td>
<td>0.049/0.689</td>
<td>0.069/0.708</td>
<td>0.078/0.702</td>
<td>0.088/0.704</td>
<td>0.102/0.711</td>
<td>0.114/0.71</td>
</tr>
<tr>
<td align="right">9</td>
<td>0.014/0.752</td>
<td>0.017/0.687</td>
<td>0.024/0.735</td>
<td>0.035/0.742</td>
<td>0.031/0.689</td>
<td>0.04/0.696</td>
<td>0.05/0.717</td>
<td>0.054/0.709</td>
<td>0.06/0.712</td>
</tr>
<tr>
<td align="right">10</td>
<td>0.006/0.755</td>
<td>0.008/0.715</td>
<td>0.013/0.753</td>
<td>0.017/0.73</td>
<td>0.018/0.723</td>
<td>0.024/0.748</td>
<td>0.024/0.716</td>
<td>0.029/0.734</td>
<td>0.034/0.731</td>
</tr>
</tbody></table>


We focus more on precision and keep a decent recall, because the purpose of our model is to obtain some nearest neighbors. If try to obtain all the nearest neigbors by increasing the r value or by decreasing the b value the number of nearest neighbors will be close to the total number of users and that defeatst the purpose os using approximate nearest neighbors to reduce the similar users set size.<br>

For the above reason we choose b = 7 and r = 10 for all the calculations ahead.

### Sim-Hash Results: <br>

Now we use the first 4 splits of the data to perform 4 fold cross validation of the the KnnBasic model using the cosine similarity as the similarity metric for the approximate nearest neigbors (refer to code : cross_validate.py). We vary the number of nearest neigbors in knn from 10-150 in intervals of 10 and calcualate the rmse and mae (refer to code: rmse.ipynb)<br>

In [3]:
from IPython.display import Image
Image(filename='plots/rmse_mae.png')

IOError: [Errno 2] No such file or directory: u'plots/rmse_mae.png'

We finally evaulate the model on a set of 6770 users and compare our knn cosine model with approximate nearest neigbors with the baseline approach and output the comparision to a plot(refer to rmse.ipynb)

In [ ]:
from IPython.display import Image
Image(filename='plots/final_plot1.png')

From the plot it is clear that out model did not outperform the baseline approch in any of the chose metrics. But we have implemented a faster approch to the knnBasic model from the project_part1 and our results are comparable to the project_part1 as shown in the figure below

In [4]:
from IPython.display import Image
Image(filename='plots/part1.png')

IOError: [Errno 2] No such file or directory: u'plots/part1.png'

In [ ]:
# Results
-----------
The following sections will explain the algorithms we used and discuss the results produced by running these algorithms

### Baseline Approach:



In [ ]:
# Conclusion
------------



# Future Considerations
-----------------------
1) Explore other features to incorporate in user/item vectors. This may include other IMDB features such as producer, writer,
and year of release, or perhaps something more complex such as tf-idf for critic reviews.
2) Combine both LSH and content-based approaches into a single ensemble system and find optimal weights for each model.
3) Use maching learning techniques to learn the features that are important for each user and thus, each user will have
personalized weights for each feature category.
4) Find the optimal number of content-based recommendations and LSH recommendations for mixed recommender system.